In [48]:
import pandas as pd
import missingno as msno
from datetime import datetime
import numpy as np
import sklearn.compose._column_transformer
import seaborn as sns
import matplotlib.pyplot as plt

In [49]:
customers = pd.read_csv("../data/olist_customers_dataset.csv") # Client
geolocalisation = pd.read_csv("../data/olist_geolocation_dataset.csv") # Loc

order_items = pd.read_csv("../data/olist_order_items_dataset.csv") # Commandes items
order_payments = pd.read_csv("../data/olist_order_payments_dataset.csv") # Commandes paiement
order_reviews = pd.read_csv("../data/olist_order_reviews_dataset.csv") # Commandes reviews
orders = pd.read_csv("../data/olist_orders_dataset.csv") # Commandes

sellers = pd.read_csv("../data/olist_sellers_dataset.csv") # Vendeurs
products = pd.read_csv("../data/olist_products_dataset.csv") # Produits
cat_trans = pd.read_csv('../data/product_category_name_translation.csv') # Translate


In [50]:
order_cust = pd.merge(left=customers, right=orders, how="left", left_on="customer_id", right_on="customer_id")
forder_cust = pd.merge(left=order_cust, right=order_items, how="outer", left_on="order_id", right_on="order_id")

forder_cust['total'] = forder_cust[["price", "freight_value"]].sum(axis=1)

## Recence 

In [51]:
dates = ['order_purchase_timestamp', 'order_approved_at','order_delivered_carrier_date', 'order_delivered_customer_date','order_estimated_delivery_date','shipping_limit_date']
forder_cust[dates] =forder_cust[dates].apply(pd.to_datetime, format='%Y-%m-%d')
date_1 = forder_cust.sort_values(by ='order_approved_at',ascending = False)


recence = date_1.groupby('customer_unique_id').agg(last_date =('order_approved_at','max')).reset_index()


recence['periode']=(date_1.order_approved_at.max()-recence.last_date).dt.days
recence.dropna(inplace=True)
recence.periode = recence.periode.astype(int)

def recence_transform(row):
    if row['periode'] <= 140:
        return 1
    if row['periode']> 140 and row['periode'] <= 280:
        return 2
    if row['periode']>280 and row['periode'] <= 420:
        return 3
    if row['periode']> 420 and row['periode'] <= 560:
        return 4
    if row['periode']> 560:
        return 5 

recence['r_value']=recence.apply(lambda row: recence_transform(row),axis=1)
recence.head()

,customer_unique_id,last_date,periode,r_value
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10 11:11:18,116,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07 18:25:44,118,1
2,0000f46a3911fa3c0805444483337064,2017-03-10 21:05:03,541,4
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-12 20:49:17,325,3
4,0004aac84e0df4da2b147fca70cf8255,2017-11-14 20:06:52,292,3


In [52]:

liste1 =cat_trans.product_category_name.tolist()
liste2 = cat_trans.product_category_name_english.tolist()
dict = {}
for key in liste1:
    for value in liste2:
        dict[key] = value
        liste2.remove(value)
        break
products.product_category_name=products.product_category_name.map(dict)
order_reviews.drop(['review_comment_title','review_comment_message'],inplace=True,axis=1)
order_reviews['review_score'].dropna(inplace=True)

## Merging pour l'analyse des commandes

### reajustement du df payments pour obtenir une ligne = une commande

In [53]:
def get_mode(x):
    return x.mode().iloc[0]

In [54]:
p1 = order_payments.groupby('order_id').agg(payment_installment = ('payment_installments',get_mode),payment_type = ('payment_type',get_mode),total_pay =('payment_value','sum')).reset_index()


### Merging products and order_items et les regrouper par commandes uniques

In [56]:
order_items.drop(['seller_id','price','freight_value'],inplace=True,axis=1)
products_item = pd.merge(order_items,products,how ='left',left_on='product_id',right_on='product_id')
a = products_item.groupby(['order_id','product_category_name']).agg(total_commande=('order_item_id','count'),prod =('product_id','nunique')).reset_index()
c = a.groupby('order_id').agg(category=('product_category_name',get_mode),items_commande = ('total_commande','sum'),prod_commande=('prod','sum')).reset_index()
df_commande = pd.merge(p1,c,how='left',left_on='order_id',right_on='order_id')
df_commande
#Dataframe montrant par commande , le type de paiement le plus utilisé, le total de la commande en R$, la catégorie du produit le plus commandé, le nb d'items commandé et le nombre de produits différents commandés


### Merging orders_items/products/payments

In [61]:
orders=orders.dropna()
orders[['order_purchase_timestamp','order_approved_at','order_delivered_customer_date']]= orders[['order_purchase_timestamp','order_approved_at','order_delivered_customer_date']].apply(pd.to_datetime)
orders['purchase_date']=orders.order_purchase_timestamp.dt.to_period('M')
orders['approval_time']= (orders['order_approved_at'] - orders['order_purchase_timestamp']).dt.days
orders['delivery_time'] = (orders['order_delivered_customer_date']-orders['order_approved_at']).dt.days
orders.delivery_time =orders.delivery_time.astype(int)
df_orders = orders[['order_id','customer_id','delivery_time','purchase_date']]

In [72]:
df_client = pd.merge(df_commande,df_orders,how='left',left_on='order_id',right_on='order_id')

In [73]:
df_client

,order_id,payment_installment,payment_type,total_pay,category,items_commande,prod_commande,customer_id,delivery_time,purchase_date
0,00010242fe8c5a6d1ba2dd792cb16214,2,credit_card,72.19,cool_stuff,1.0,1.0,3ce436f183e68e07877b285a838db11a,7.0,2017-09
1,00018f77f2f0320c557190d7a144bdd3,3,credit_card,259.83,pet_shop,1.0,1.0,f6dd3ec061db4e3987629fe6b26e5cce,16.0,2017-04
2,000229ec398224ef6ca0657da4fc703e,5,credit_card,216.87,furniture_decor,1.0,1.0,6489ae5e4333f3693df5ad4372dab6d3,7.0,2018-01
3,00024acbcdf0a6daa1e931b038114c75,2,credit_card,25.78,perfumery,1.0,1.0,d4eb9395c8c0431ee92fce09860c5a06,6.0,2018-08
4,00042b26cf59d7ce69dfabb4e55b4fd9,3,credit_card,218.04,garden_tools,1.0,1.0,58dbd0b2d70206bf40e62cd34e84d795,25.0,2017-02
...,...,...,...,...,...,...,...,...,...,...
99435,fffc94f6ce00a00581880bf54a75a037,1,boleto,343.40,housewares,1.0,1.0,b51593916b4b8e0d6f66f2ae24f2673d,15.0,2018-04
99436,fffcd46ef2263f404302a634eb57f7eb,1,boleto,386.53,computers_accessories,1.0,1.0,84c5d4fbaf120aae381fad077416eaa0,6.0,2018-07
99437,fffce4705a9662cd70adb13d4a31832d,3,credit_card,116.85,sports_leisure,1.0,1.0,29309aa813182aaddc9b259e31b870e6,3.0,2017-10
99438,fffe18544ffabc95dfada21779c9644f,3,credit_card,64.71,computers_accessories,1.0,1.0,b5e6afd5a41800fdf401e0272ca74655,1.0,2017-08


In [104]:
geolocalisation.geolocation_zip_code_prefix= geolocalisation.geolocation_zip_code_prefix.astype(str)

In [106]:
customers.customer_zip_code_prefix= customers.customer_zip_code_prefix.astype(str)

In [107]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  object
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: object(5)
memory usage: 3.8+ MB


In [110]:
geolocalisation[geolocalisation.geolocation_zip_code_prefix=='1037']

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
14,1037,-23.545187,-46.637855,são paulo,SP
31,1037,-23.546705,-46.640336,são paulo,SP
169,1037,-23.543883,-46.638075,são paulo,SP
178,1037,-23.546157,-46.639885,sao paulo,SP
236,1037,-23.543883,-46.638075,sao paulo,SP
283,1037,-23.545199,-46.637916,sao paulo,SP
417,1037,-23.545187,-46.637855,sao paulo,SP
420,1037,-23.546723,-46.640281,sao paulo,SP
433,1037,-23.546463,-46.640145,sao paulo,SP


In [108]:
df_custo = customers.merge(geolocalisation,how='left',left_on='customer_zip_code_prefix',right_on='geolocation_zip_code_prefix')
df_custo

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,14409,-20.509897,-47.397866,franca,SP
1,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,14409,-20.497396,-47.399241,franca,SP
2,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,14409,-20.510459,-47.399553,franca,SP
3,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,14409,-20.480940,-47.394161,franca,SP
4,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,14409,-20.515413,-47.398194,franca,SP
...,...,...,...,...,...,...,...,...,...,...
15083728,274fa6071e5e17fe303b9748641082c8,84732c5050c01db9b23e19ba39899398,6703,cotia,SP,6703,-23.599369,-46.905603,cotia,SP
15083729,274fa6071e5e17fe303b9748641082c8,84732c5050c01db9b23e19ba39899398,6703,cotia,SP,6703,-23.593577,-46.910112,cotia,SP
15083730,274fa6071e5e17fe303b9748641082c8,84732c5050c01db9b23e19ba39899398,6703,cotia,SP,6703,-23.584425,-46.892014,cotia,SP
15083731,274fa6071e5e17fe303b9748641082c8,84732c5050c01db9b23e19ba39899398,6703,cotia,SP,6703,-23.595022,-46.918546,cotia,SP


In [90]:
sellers.seller_state.nunique()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP
...,...,...,...,...
3090,98dddbc4601dd4443ca174359b237166,87111,sarandi,PR
3091,f8201cab383e484733266d1906e2fdfa,88137,palhoca,SC
3092,74871d19219c7d518d0090283e03c137,4650,sao paulo,SP
3093,e603cf3fec55f8697c9059638d6c8eb5,96080,pelotas,RS
